In [1]:
import google.generativeai as genai
from google.generativeai.types import GenerationConfig
import os
import time
import csv
import pandas as pd
import matplotlib.pyplot as plt
import sacrebleu
from src.Generation.Equation_BaseDataset import BaseDataset

c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
api_key = os.getenv("GEMINI_API_KEY")  # Returns None if the variable is not set
# Check if the API key is loaded correctly
if api_key:
    print("API key loaded successfully!")
else:
    print("API key not found. Please ensure it is set as an environment variable.")

genai.configure(api_key=api_key)

API key loaded successfully!


In [6]:
class GeminiDataset(BaseDataset):
    def __init__(self, num, filepath):
        super().__init__(num, filepath)

    def get_columns(self):
        return ["text_input", "output"]

In [12]:
# Path to CSV file
csv_file_path = "Data/gemini_test_6.csv"

data = GeminiDataset(100, csv_file_path)
data.create()

Generating dataset: 100%|██████████| 100/100 [00:01<00:00, 94.95it/s]


In [9]:
dataset = data.dataset
print(dataset[0])

{'text_input': '<mml:mi>l</mml:mi>\n<mml:mo>=</mml:mo>\n<mml:mrow>\n<mml:mn>3</mml:mn>\n<mml:mrow>\n<mml:mi>sin</mml:mi>\n<mml:mfenced>\n<mml:msub>\n<mml:mi>w</mml:mi>\n<mml:mrow>\n<mml:mi>υ</mml:mi>\n<mml:mi>z</mml:mi>\n</mml:mrow>\n</mml:msub>\n</mml:mfenced>\n</mml:mrow>\n</mml:mrow>', 'output': "l = Symbol('l')\nw_υz = Symbol('w_υz')\ne = Eq(l, 3*sin(w_υz))"}


In [6]:
for model in genai.list_tuned_models():
    print(model.name)    

# for model in genai.list_models():
#     print(model.name)   

tunedModels/mmltopython1-si6b5lpywfhi
tunedModels/mmltopython2-rq77o9de7x0p
tunedModels/mmltopython4-f3fuppiemnq9
tunedModels/mmltopython5-t7qh9rtx0wwt
tunedModels/mmltopython6-i1nlb0w0glxy


In [4]:
base_model = "models/gemini-1.5-flash-001-tuning"

operation = genai.create_tuned_model(
    display_name="mml_to_python_6",
    source_model=base_model,
    epoch_count=4,
    batch_size=16,
    learning_rate=0.002,
    training_data=dataset,
)

for status in operation.wait_bar():
    time.sleep(10)

result = operation.result()
print(result)

snapshots = pd.DataFrame(result.tuning_task.snapshots)
plt.plot(snapshots['epoch'], snapshots['mean_loss'])

NameError: name 'dataset' is not defined

In [9]:
tuned_model = 'tunedModels/mmltopython1-si6b5lpywfhi'
model = genai.GenerativeModel(model_name=tuned_model)
generation_config = GenerationConfig(temperature=0.1)

# mml = "\n<mml:mrow>\n<mml:mrow>\n<mml:mo>-</mml:mo>\n<mml:msqrt>\n<mml:mi>y</mml:mi>\n</mml:msqrt>\n</mml:mrow>\n<mml:mo>+</mml:mo>\n<mml:mrow>\n<mml:mi>cos</mml:mi>\n<mml:mfenced>\n<mml:msub>\n<mml:mi>Υ</mml:mi>\n<mml:mi>ξ</mml:mi>\n</mml:msub>\n</mml:mfenced>\n</mml:mrow>\n</mml:mrow>\n<mml:mo>=</mml:mo>\n<mml:mrow>\n<mml:mrow>\n<mml:msub>\n<mml:mi>Ψ</mml:mi>\n<mml:mi>G</mml:mi>\n</mml:msub>\n<mml:mrow>\n<mml:mi>log</mml:mi>\n<mml:mfenced>\n<mml:mi>M</mml:mi>\n</mml:mfenced>\n</mml:mrow>\n</mml:mrow>\n<mml:mo>-</mml:mo>\n<mml:mrow>\n<mml:mi>sin</mml:mi>\n<mml:mfenced>\n<mml:mrow>\n<mml:mn>2</mml:mn>\n<mml:mi>k</mml:mi>\n</mml:mrow>\n</mml:mfenced>\n</mml:mrow>\n</mml:mrow>\n"

prompt = f"""
<mml:mi>h</mml:mi>
  <mml:mo>=</mml:mo>
  <mml:mrow>
    <mml:msub>
      <mml:mi>h</mml:mi>
      <mml:mi>c</mml:mi>
    </mml:msub>
    <mml:mo>+</mml:mo>
    <mml:msub>
      <mml:mi>h</mml:mi>
      <mml:mi>g</mml:mi>
    </mml:msub>
  </mml:mrow>
"""

response = model.generate_content(prompt, generation_config=generation_config)

print(response.text)


h = Symbol('h')
h_c = Symbol('h_c')
h_g = Symbol('h_g')
e = Eq(h, h_c + h_g)


In [1]:
import csv
def gemini_translate(filename, model_name):   
    predictions = []  
    i = 0
    with open(f"{filename}.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            mml = row[0]
            print(mml)
            prompt = f"""
            You are an expert at translating MathML to Python code using the SymPy library. Translate the following MathML expression into a valid Python SymPy expression. 
            Ensure that all variables are treated as symbolic variables and each is defined separately on a new line. 
            Assume that SymPy has been imported as `from sympy import *` so DO NOT include any import statements in your output.
            Example: 
            MathML: '<mml:mi>p</mml:mi>\n<mml:mo>=</mml:mo>\n<mml:mrow>\n<mml:msub>\n<mml:mi>p</mml:mi>\n<mml:mi>a</mml:mi>\n</mml:msub>\n<mml:mo>-</mml:mo>\n<mml:msub>\n<mml:mi>y</mml:mi>\n<mml:mrow>\n<mml:mi>r</mml:mi>\n<mml:mi>e</mml:mi>\n<mml:mi>2</mml:mi>\n</mml:mrow>\n</mml:msub>\n</mml:mrow>'
            SymPy Python code: 'p = Symbols('p')\np_a = Symbols('p_a')\ny_re2 = Symbols('y_re2')\ne = Eq(p, p_a - y_re2)'
            
            Now, translate the following MathML expression into Python code using SymPy.
            MathML: {mml}
            SymPy Python code:
            """
            model = genai.GenerativeModel(model_name=model_name)
            generation_config = GenerationConfig(temperature=0.1)
            response = model.generate_content(prompt, generation_config=generation_config)
            text = response.text
            predictions.append(text)
            i += 1
            print(f"Generated {i} predictions")
    with open(f"{filename}_predictions.csv", "w", newline="") as file:
        writer = csv.writer(file)
        for pred in predictions:
            writer.writerow([pred])

def evaluate_bleu(filename, model_name):
    gemini_translate(filename, model_name)

    with open(f"{filename}.csv", "r") as file:
        reader = csv.reader(file)
        next(reader)
        refs = [row[1] for row in reader]
    with open(f"{filename}_predictions.csv", "r") as file:
        reader = csv.reader(file)
        preds = [row[0] for row in reader]

    bleu = sacrebleu.corpus_bleu(preds, [refs])
    print(f"SacreBLEU Score: {bleu.score}")

# evaluate_bleu("gemini_test_1", 'tunedModels/mmltopython1-si6b5lpywfhi')
evaluate_bleu("Data/gemini_test_6", "tunedModels/mmltopython6-i1nlb0w0glxy")

<mml:mi>m</mml:mi>
<mml:mo>=</mml:mo>
<mml:mrow>
<mml:mrow>
<mml:mi>cos</mml:mi>
<mml:mfenced>
<mml:mi>y</mml:mi>
</mml:mfenced>
</mml:mrow>
<mml:mo>+</mml:mo>
<mml:mn>3</mml:mn>
</mml:mrow>


NameError: name 'genai' is not defined